# Command visualization app

Note: this notebook is also available in [Google Colaboratoy](https://colab.research.google.com/github/paranal-sw/collaborations-UFRO-2024-2s/blob/main/1-command-visualization/task_description.ipynb)

The VLT Software has a mechanism to send command between processes. This is normally started in the instrument (PIONIER, MATISSE, GRAVITY) and in some cases like template ACQ or template OBS, the command is sent to subsystems (ISS, DL, ATx, ARAL, etc)

It would be useful to have a graphical tool to understands who initiated the messages, the trip it makes through the systems, and included their parameters whenever is possible.

## Objective
1. Create a streamlit app to display command messages through processes.
1. Fill the gaps when commands send and receive are not clearly written.
1. Report the process to create the app and the assumtpioms made to fill the gaps.

Make a UML sequence diagram with the time(start and end) between process(if it fails or not), the instrument, the command(name) and the susbsystem(name), 

## Example app

A sample streamlit app is provided to use as barebones. Please create subpages with different versions.

To start the app:

```bash
cd collaborations-UFRO-2024-2s
source venv/bin/activate

cd 1-command-visualization
streamlit run Home_example1.py
```

In [1]:
import os 
import pandas as pd
from urllib.request import urlretrieve
REPO_URL='https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    
    PATH='data/' # Convenient name to be Colab compatible
else:
    PATH='D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data' # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):

    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst=pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs=pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele=pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[ df_all['trace_id']==trace_id ]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta=pd.read_parquet(f'{PATH}/{fname}')

    return df_meta

## Example of command events

In [35]:
df_meta = load_meta("MATISSE", "1d")
df_meta

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,SECONDS,TEL
0,2019-04-10 00:16:03.269,2019-04-10 00:20:35.831,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,272.0,AT
1,2019-04-10 00:20:35.848,2019-04-10 00:42:17.085,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,1301.0,AT
2,2019-04-10 00:42:17.106,2019-04-10 01:11:09.300,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,1732.0,AT
3,2019-04-10 01:11:09.325,2019-04-10 01:22:27.609,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,678.0,AT
4,2019-04-10 01:22:40.579,2019-04-10 01:29:34.127,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,413.0,AT
...,...,...,...,...,...,...,...,...,...,...
65,2019-04-10 23:36:34.218,2019-04-10 23:37:22.734,False,MATISSE,bob_163744,MATISSE_img_acq,False,False,48.0,AT
66,2019-04-10 23:37:45.605,2019-04-10 23:40:12.335,False,MATISSE,bob_163744,MATISSE_img_acq,False,False,146.0,AT
67,2019-04-10 23:40:15.899,2019-04-10 23:43:40.407,False,MATISSE,bob_163744,MATISSE_img_acq,False,False,204.0,AT
68,2019-04-10 23:45:18.254,2019-04-10 23:51:49.993,False,MATISSE,bob_163744,MATISSE_img_acq,False,False,391.0,AT


In [39]:
df_meta['Aborted'].value_counts()

Aborted
False    69
True      1
Name: count, dtype: int64

In [ ]:
df_meta['ERROR'].value_counts()

In [40]:
df_meta['TIMEOUT'].value_counts()

TIMEOUT
False    70
Name: count, dtype: int64

In [3]:
df_trace = load_trace("MATISSE", "1d", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]

### Search *command*


In [4]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("command") ][:50])

,@timestamp,system,procname,logtext,trace_id
95,2019-04-10 00:16:04.309,ISS,issifControl,"CDP : Received command: GETCONF, Buffer:",0
96,2019-04-10 00:16:04.311,ISS,issifControl,"CDP : Succesfully completed command: GETCONF, Buffer: IRIS,#012SINGLE_STS,#0121,AT1,A0,DL6,1,compressed,147.596,12.545,8.323,#0122,AT2,B2,DL5,3,compressed,163.345,",0
97,2019-04-10 00:16:04.355,ISS,issifControl,"CDP : Received command: GETWCS, Buffer: 2",0
98,2019-04-10 00:16:04.356,ISS,issifControl,"CDP : Succesfully completed command: GETWCS, Buffer: 109.144148,-27.880972,270.111594",0
99,2019-04-10 00:16:04.401,ISS,issifControl,"CDP : Received command: GETWCS, Buffer: 1",0
100,2019-04-10 00:16:04.401,ISS,issifControl,"CDP : Succesfully completed command: GETWCS, Buffer: 109.144148,-27.880972,270.111594",0
104,2019-04-10 00:16:04.447,ISS,issifControl,"CDP : Received command: GETWCS, Buffer: 3",0
105,2019-04-10 00:16:04.448,ISS,issifControl,"CDP : Succesfully completed command: GETWCS, Buffer: 109.144148,-27.880972,270.111594",0
108,2019-04-10 00:16:04.493,ISS,issifControl,"CDP : Received command: GETWCS, Buffer: 4",0
109,2019-04-10 00:16:04.494,ISS,issifControl,"CDP : Succesfully completed command: GETWCS, Buffer: 109.144149,-27.881805,270.111594",0


### List of commands

This method must be double checked

In [5]:
cmd = df_trace[ df_trace['logtext'].str.contains("command")]['logtext']

# Please replace this with an elegant regexp!
cmd = cmd.apply(lambda x: x.replace(":", " "))
cmd = cmd.apply(lambda x: x.replace(",", " "))
cmd = cmd.apply(lambda x: x.replace("'", " "))
cmd = cmd.apply(lambda x: x.replace("  ", " "))

cmdlist = []
for x in cmd:
    splitted = x.split()
    # print(splitted)
    if "command" in splitted:
        idx = splitted.index("command")
        command = splitted[idx+1]
        if command not in ["...", "done.", "to"]:
            cmdlist.append(command)
set(cmdlist)

{'CHKMOON',
 'CLRSTP',
 'GETCONF',
 'GETDELD',
 'GETIAS',
 'GETWCS',
 'PRABS',
 'PRCHK',
 'PRESET',
 'PRSNAME',
 'PRTOGS',
 'SETBEAM',
 'SETPOF',
 'SETREF',
 'SETTADC',
 'SETUP',
 'STOPAG',
 'STOPCAG',
 'STOPCHP',
 'STOPLAG',
 'STRTLAG'}

### Search *SETUP*


In [6]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("SETUP") ][:50])

,@timestamp,system,procname,logtext,trace_id
151,2019-04-10 00:16:04.931,MATISSE,bob_ins,SETUP -noExposure -function DET1.START.VAL 1 DET2.START.VAL 1 (blue),0
152,2019-04-10 00:16:04.932,MATISSE,mtoControl,Executing SETUP command ...,0
153,2019-04-10 00:16:04.933,MATISSE,mtoControl,Forward(b) SETUP to NGCIR2,0
154,2019-04-10 00:16:04.933,MATISSE,mtoControl,Forward(b) SETUP to NGCIR1,0
155,2019-04-10 00:16:04.936,MATISSE,mtoControl,SETUP command done.,0
159,2019-04-10 00:16:05.071,MATISSE,mtoControl,Executing SETUP command ...,0
160,2019-04-10 00:16:05.071,MATISSE,bob_ins,SETUP -expoId 0 -function INS.MODE HYBRID (blue),0
161,2019-04-10 00:16:05.879,MATISSE,mtoControl,SETUP command done.,0
163,2019-04-10 00:16:05.880,MATISSE,mtoControl,Executing SETUP command ...,0
164,2019-04-10 00:16:05.880,MATISSE,bob_ins,SETUP -expoId 146 -function INS.MODE HYBRID INS.BSL1.ST T INS.BSL2.ST T INS.BSL3.ST T INS.BSL4.ST T (blue),0


### Search *PRESET*


In [7]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("PRESET") ][:59])

,@timestamp,system,procname,logtext,trace_id
65,2019-04-10 00:16:03.534,MATISSE,bob_ins,SEQ PRESET ST = 'T',0
266,2019-04-10 00:16:15.793,AT1,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
268,2019-04-10 00:16:15.794,AT2,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
269,2019-04-10 00:16:15.795,AT3,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
272,2019-04-10 00:16:15.796,AT4,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
275,2019-04-10 00:16:15.797,DL,dlprsControl,"PRESET: Received command: CLRSTP, Buffer:",0
276,2019-04-10 00:16:15.801,DL,dlprsControl,PRESET: Command CLRSTP (NoMove) succesfully completed,0
279,2019-04-10 00:16:15.843,AT1,prsControl,PRESET: Command STOPTRK (NoMove) succesfully completed,0
281,2019-04-10 00:16:15.859,AT3,prsControl,PRESET: Command STOPTRK (NoMove) succesfully completed,0
283,2019-04-10 00:16:15.873,AT2,prsControl,PRESET: Command STOPTRK (NoMove) succesfully completed,0


In [33]:
import pandas as pd
# the location of the files data
file_path = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data/PIONIER-1w-meta.parket'
# read the dataframe from the parquet file
df = pd.read_parquet(file_path)
df



,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,SECONDS,TEL
0,2019-04-01 22:29:07.746,2019-04-01 22:29:10.519,False,PIONIER,bob_ins,PIONIER_gen_tec_setup,False,False,2.0,AT
1,2019-04-02 07:40:48.591,2019-04-02 07:41:21.459,False,PIONIER,bob_25299,PIONIER_gen_tec_setup,False,False,32.0,AT
2,2019-04-02 07:41:21.488,2019-04-02 07:42:52.294,False,PIONIER,bob_25299,PIONIER_gen_tec_niobate,False,False,90.0,AT
3,2019-04-02 07:42:52.314,2019-04-02 07:43:24.984,False,PIONIER,bob_25299,PIONIER_gen_tec_setup,False,False,32.0,AT
4,2019-04-02 09:21:52.714,2019-04-02 09:22:25.281,False,PIONIER,bob_ins,PIONIER_gen_tec_setup,False,False,32.0,AT
...,...,...,...,...,...,...,...,...,...,...
389,2019-04-07 20:37:40.154,2019-04-07 20:38:04.954,False,PIONIER,bob_ins,PIONIER_gen_tec_setup,False,False,24.0,AT
390,2019-04-07 20:38:04.980,2019-04-07 20:39:04.254,False,PIONIER,bob_ins,PIONIER_gen_tec_niobate,False,False,59.0,AT
391,2019-04-07 20:39:04.272,2019-04-07 20:39:29.141,False,PIONIER,bob_ins,PIONIER_gen_tec_setup,False,False,24.0,AT
392,2019-04-07 20:39:29.165,2019-04-07 20:40:03.846,False,PIONIER,bob_ins,PIONIER_gen_tec_standby,False,False,34.0,AT


In [34]:
df_meta = load_meta("PIONIER", "1w")
df_meta

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,SECONDS,TEL
0,2019-04-01 22:29:07.746,2019-04-01 22:29:10.519,False,PIONIER,bob_ins,PIONIER_gen_tec_setup,False,False,2.0,AT
1,2019-04-02 07:40:48.591,2019-04-02 07:41:21.459,False,PIONIER,bob_25299,PIONIER_gen_tec_setup,False,False,32.0,AT
2,2019-04-02 07:41:21.488,2019-04-02 07:42:52.294,False,PIONIER,bob_25299,PIONIER_gen_tec_niobate,False,False,90.0,AT
3,2019-04-02 07:42:52.314,2019-04-02 07:43:24.984,False,PIONIER,bob_25299,PIONIER_gen_tec_setup,False,False,32.0,AT
4,2019-04-02 09:21:52.714,2019-04-02 09:22:25.281,False,PIONIER,bob_ins,PIONIER_gen_tec_setup,False,False,32.0,AT
...,...,...,...,...,...,...,...,...,...,...
389,2019-04-07 20:37:40.154,2019-04-07 20:38:04.954,False,PIONIER,bob_ins,PIONIER_gen_tec_setup,False,False,24.0,AT
390,2019-04-07 20:38:04.980,2019-04-07 20:39:04.254,False,PIONIER,bob_ins,PIONIER_gen_tec_niobate,False,False,59.0,AT
391,2019-04-07 20:39:04.272,2019-04-07 20:39:29.141,False,PIONIER,bob_ins,PIONIER_gen_tec_setup,False,False,24.0,AT
392,2019-04-07 20:39:29.165,2019-04-07 20:40:03.846,False,PIONIER,bob_ins,PIONIER_gen_tec_standby,False,False,34.0,AT
